In [2]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.absolute()))
import config
import time
from context import Context
from multiversx_sdk.core import Address, AddressComputer
from multiversx_sdk.core import SmartContractTransactionsFactory
from multiversx_sdk.core import TransactionsFactoryConfig
from multiversx_sdk import ProxyNetworkProvider
from multiversx_sdk.core import TokenPayment, TokenComputer
from utils.utils_chain import WrapperAddress as Address, Account
from utils.utils_tx import endpoint_call
from contracts.price_discovery_contract import PriceDiscoveryContract
from multiversx_sdk.abi import Abi

context = Context()


proxy = ProxyNetworkProvider(config.devnet)
network_config = proxy.get_network_config()
net = "devnet" if "devnet" in proxy.url else "testnet" if "testnet" in proxy.url else "mainnet"
abi = Abi.load(Path(
    "/home/multiversx/Documents/mock_fwk/wasm/price-discovery/price-discovery.abi.json"
))
factory = SmartContractTransactionsFactory(TransactionsFactoryConfig(network_config.chain_id), abi)

factory_config = TransactionsFactoryConfig(network_config.chain_id)
# transactions_factory = SmartContractTransactionsFactory(factory_config, abi)



2025-02-12 12:04:35,318 - WARNING - Structure definition for stakings_boosted not found in /home/multiversx/Documents/mock_fwk/dex_deploy/configs-mainnet/deploy_structure.json


loaded 1 accounts from 1 PEM files.


2025-02-12 12:04:35,569 - DEBUG - Account.sync_nonce() done: 0
2025-02-12 12:04:35,657 - DEBUG - Account.sync_nonce() done: 0
2025-02-12 12:04:35,658 - INFO - Loaded 10 tokens.
2025-02-12 12:04:35,745 - DEBUG - Account.sync_nonce() done: 0
2025-02-12 12:04:35,745 - INFO - Loaded 0 stakings_boosted.
2025-02-12 12:04:35,844 - DEBUG - Account.sync_nonce() done: 0
2025-02-12 12:04:35,845 - INFO - Deployed contracts below:
2025-02-12 12:04:35,845 - INFO - stakings_boosted:


# SETUP

In [3]:
launched_token_id = "TKNZPWA-f0763d"
accepted_token_id = "TKNZPWA-a8b1bc"

user_min_deposit: int = 900000
admin = "erd1zpwadzzylule6njjtrrhf0hcysak23a76983tq6j7hddn66edr6q9qjytp"

deployer = Account(pem_file=config.DEFAULT_OWNER) 
deployer.sync_nonce(context.network_provider.proxy)
deployer_shard = AddressComputer().get_shard_of_address(deployer.address)



2025-02-12 12:04:43,591 - DEBUG - Account.sync_nonce() done: 5605


In [4]:
from utils import blocktime


start_time: int = "25/02/12 12:30"
user_deposit_withdraw_time: int = "25/02/12 16:00"
owner_deposit_withdraw_time: int = "25/02/12 17:15"

print(f"Start round calculation below:")
_, start_round , start_timestamp = blocktime.main([f"--net={net}", f"--shard={deployer_shard}", f"--date={start_time}"])
print(start_round)

print("===========================================")
print(f"User deposit end round calculation below:")
_, start_round, user_deposit_timestamp = blocktime.main([f"--net={net}", f"--shard={deployer_shard}", f"--date={user_deposit_withdraw_time}"])
print(start_round)

print("===========================================")
print(f"Owner deposit end round calculation below:")
_, start_round, owner_deposit_timestamp = blocktime.main([f"--net={net}", f"--shard={deployer_shard}", f"--date={owner_deposit_withdraw_time}"])

print(start_round)

Start round calculation below:
Calculating from block: 7488101 round: 7559114 shard: 0
Current date: 2025-02-12 12:04:44 (timestamp of last produced block)
Target date: 2025-02-12 12:30:00
Target time: 1739356200
7559366
User deposit end round calculation below:
Calculating from block: 7488101 round: 7559114 shard: 0
Current date: 2025-02-12 12:04:44 (timestamp of last produced block)
Target date: 2025-02-12 16:00:00
Target time: 1739368800
7561466
Owner deposit end round calculation below:
Calculating from block: 7488101 round: 7559114 shard: 0
Current date: 2025-02-12 12:04:44 (timestamp of last produced block)
Target date: 2025-02-12 17:15:00
Target time: 1739373300
7562216


# DEPLOY

In [5]:
contract = PriceDiscoveryContract(launched_token_id, accepted_token_id,"",start_timestamp, user_deposit_timestamp, owner_deposit_timestamp,user_min_deposit,admin)
args = [
        launched_token_id, 
        accepted_token_id,
        18,
        start_timestamp,
        user_deposit_timestamp,
        owner_deposit_timestamp,
        user_min_deposit,
        deployer.address
        ]
# provider = ProxyNetworkProvider(config.DEFAULT_PROXY) 
# deployer.address = Address.from_bech32("erd1wwx5zhmx9mag9k8zuajj8c9zce6mzerfr3m9ck6l7c00dwkq2a2ssyfnzk")
# tx = contract.contract_deploy(deployer, context.network_provider.proxy, config.PRICE_DISCOVERY_BYTECODE_PATH, [])
# tx = "1b3dacf1507ef888a8f1bd86e2749d23ec7d3b83106f7cb608a961bb465116ba"
# tx_status = context.network_provider.check_simple_tx_status(tx)

tx = factory.create_transaction_for_deploy(deployer.address, config.PRICE_DISCOVERY_BYTECODE_PATH, 150000000, arguments=args)
tx.nonce = deployer.nonce
tx.signature = deployer.sign_transaction(tx)

hash = proxy.send_transaction(tx)
deployer.nonce += 1
time.sleep(6)
if proxy.get_transaction_status(hash).is_failed():
    raise Exception(f"Transaction failed: {hash}")

receipt = proxy.get_transaction(hash)
contract_address = receipt.logs.find_first_or_none_event("SCDeploy").address

print(f"Contract deployed at address: {contract_address.to_bech32()}")

contract.address = contract_address.to_bech32()


Contract deployed at address: erd1qqqqqqqqqqqqqpgqg9s9zp4ly82f6g9lhy2nm29uzf9uw87hdr6qsp5wvp


ISSUE REDEEM TOKENS

In [11]:
redeem_token_name = "REDEEM"

In [12]:
contract = PriceDiscoveryContract(launched_token_id, accepted_token_id,"",start_timestamp, user_deposit_timestamp, owner_deposit_timestamp,user_min_deposit,admin)
# contract.address = "erd1qqqqqqqqqqqqqpgqg9s9zp4ly82f6g9lhy2nm29uzf9uw87hdr6qsp5wvp"
contract.print_contract_info()

contract.issue_redeem_token(deployer, proxy, redeem_token_name)
# print(redeem_token)

2025-02-12 12:12:31,583 - INFO - Deployed price discovery contract: NotImplemented
2025-02-12 12:12:31,584 - INFO -  ├ Redeem token: 
2025-02-12 12:12:31,584 - INFO -  ├ Start time: 1739356200
2025-02-12 12:12:31,585 - INFO - Issue price discovery redeem token


TypeError: 'NotImplementedType' object is not iterable

SET TRANSFER ROLE

In [ ]:
from multiversx_sdk import Address, TransactionComputer

user = Account(pem_file="/home/multiversx/Documents/mock_fwk/wallet1.pem")
user.sync_nonce(proxy)
contract_address=Address.new_from_bech32("erd1qqqqqqqqqqqqqpgqpwe5z60vqmnjluq7cqzdrs3gvsghsy09dr6qt8v32z")

set_transfer_role_farm_token_tx = factory.create_transaction_for_execute(
    sender=user.address,
    contract=contract_address,
    function="setTransferRole",
    gas_limit=70000000,
)

set_transfer_role_farm_token_tx.nonce = user.nonce
set_transfer_role_farm_token_tx.signature = user.signer.sign(
    TransactionComputer().compute_bytes_for_signing(set_transfer_role_farm_token_tx)
)

result = proxy.send_transaction(set_transfer_role_farm_token_tx)

2025-02-10 15:53:33,998 - DEBUG - Account.sync_nonce() done: 5599


SET MIN LAUNCHED TOKENS

In [35]:
from multiversx_sdk import Address, TransactionComputer

user = Account(pem_file="/home/multiversx/Documents/mock_fwk/wallet1.pem")
user.sync_nonce(proxy)
amount = 10000 * 10 ** 18
contract_address=Address.new_from_bech32("erd1qqqqqqqqqqqqqpgqpwe5z60vqmnjluq7cqzdrs3gvsghsy09dr6qt8v32z")
# event = DepositPDLiquidityEvent(launched_token_id, amount)



set_min_launched_tx = factory.create_transaction_for_execute(
    sender=user.address,
    contract=contract_address,
    function="setMinLaunchedTokens",
    gas_limit=70000000,
    arguments = [amount]
)

set_min_launched_tx.nonce = user.nonce
set_min_launched_tx.signature = user.signer.sign(
    TransactionComputer().compute_bytes_for_signing(set_min_launched_tx)
)

result = proxy.send_transaction(set_min_launched_tx)

2025-02-10 15:57:18,270 - DEBUG - Account.sync_nonce() done: 5601


CHANGE OWNER

In [7]:
second_owner = Account(pem_file=config.DEFAULT_ADMIN)
second_owner.sync_nonce(proxy)

first_owner = Account(pem_file=config.DEFAULT_OWNER)
first_owner.sync_nonce(proxy)

user = Account(pem_file=config.DEFAULT_ACCOUNTS)
user.address = Address("erd1zpwadzzylule6njjtrrhf0hcysak23a76983tq6j7hddn66edr6q9qjytp")
user.sync_nonce(proxy)

2025-02-07 07:28:54,967 - DEBUG - Account.sync_nonce() done: 956
2025-02-07 07:28:55,064 - DEBUG - Account.sync_nonce() done: 5589
2025-02-07 07:28:55,236 - DEBUG - Account.sync_nonce() done: 5589


In [ ]:
hash = endpoint_call(proxy, 50000000, second_owner, contract_address, 
                     "ChangeOwnerAddress", [first_owner.address])

# WHITELIST

In [11]:
whitelisted_address = "erd1my8mcw2fhhsvgtzah7ntr9sdm9zxlamul6uhksyfcvzagemwad7qprplwz"
deployer.sync_nonce(proxy)
contract.add_user_to_whitelist(proxy, deployer, whitelisted_address)

2025-02-10 15:33:32,127 - DEBUG - Account.sync_nonce() done: 5597
2025-02-10 15:33:32,127 - INFO - Add user to whitelist
2025-02-10 15:33:32,128 - DEBUG - Account: erd1zpwadzzylule6njjtrrhf0hcysak23a76983tq6j7hddn66edr6q9qjytp


# DEPOSIT

In [36]:
from events.price_discovery_events import DepositPDLiquidityEvent

user = Account(pem_file="/home/multiversx/Documents/mock_fwk/wallet1.pem")
user.sync_nonce(proxy)
amount = 10000 * 10 ** 18
event = DepositPDLiquidityEvent(launched_token_id, amount)

contract.deposit_liquidity(context.network_provider.proxy, user, event)

2025-02-10 15:57:38,220 - DEBUG - Account.sync_nonce() done: 5602
2025-02-10 15:57:38,221 - INFO - Deposit Price Discovery liquidity
2025-02-10 15:57:38,221 - DEBUG - Account: erd1zpwadzzylule6njjtrrhf0hcysak23a76983tq6j7hddn66edr6q9qjytp
2025-02-10 15:57:38,222 - DEBUG - Token: TKNZPWA-f0763d Amount: 10000000000000000000000
2025-02-10 15:57:38,222 - DEBUG - Deposit Price Discovery liquidity
2025-02-10 15:57:38,309 - DEBUG - Contract call arguments: []
The transaction builders are deprecated and will be removed in a later release. Use the transaction factories instead.
2025-02-10 15:57:38,423 - INFO - View this transaction in the MultiversX Devnet Explorer: https://devnet-explorer.multiversx.com/transactions/173521d8aeaf7e29f2978ca23316980cc26a495e9ff7828021a7a567cc6d3560


'173521d8aeaf7e29f2978ca23316980cc26a495e9ff7828021a7a567cc6d3560'

USER DEPOSIT

In [11]:
from events.price_discovery_events import DepositPDLiquidityEvent

user = Account(pem_file="/home/multiversx/Documents/mock_fwk/wallet2.pem")
user.sync_nonce(proxy)
amount = 10000 * 10 ** 18
event = DepositPDLiquidityEvent(accepted_token_id, amount)

contract.user_deposit(context.network_provider.proxy, user, event)

2025-02-10 16:12:03,220 - DEBUG - Account.sync_nonce() done: 1031
2025-02-10 16:12:03,221 - INFO - Deposit accepted token
2025-02-10 16:12:03,222 - DEBUG - Account: erd1my8mcw2fhhsvgtzah7ntr9sdm9zxlamul6uhksyfcvzagemwad7qprplwz
2025-02-10 16:12:03,222 - DEBUG - Token: TKNZPWA-a8b1bc Amount: 10000000000000000000000
2025-02-10 16:12:03,223 - DEBUG - Deposit accepted token
2025-02-10 16:12:03,387 - DEBUG - Contract call arguments: []
The transaction builders are deprecated and will be removed in a later release. Use the transaction factories instead.
2025-02-10 16:12:03,499 - INFO - View this transaction in the MultiversX Devnet Explorer: https://devnet-explorer.multiversx.com/transactions/97f34100860adca6f2b09944f9f984b2a8500ea60aa79856fca494ed416974a1


'97f34100860adca6f2b09944f9f984b2a8500ea60aa79856fca494ed416974a1'

USER WITHDRAW

In [12]:
from events.price_discovery_events import WithdrawPDLiquidityEvent
# contract.address=Address.new_from_bech32("erd1qqqqqqqqqqqqqpgqpwe5z60vqmnjluq7cqzdrs3gvsghsy09dr6qt8v32z")

user = Account(pem_file="/home/multiversx/Documents/mock_fwk/wallet2.pem")
user.sync_nonce(proxy)
amount = 20000 * 10 ** 18


event = WithdrawPDLiquidityEvent("REDEEM-46eed7",0, amount)

contract.user_withdraw(context.network_provider.proxy, user, event)

2025-02-10 16:12:31,535 - DEBUG - Account.sync_nonce() done: 1032
2025-02-10 16:12:31,536 - INFO - Withdraw accepted token
2025-02-10 16:12:31,536 - DEBUG - Account: erd1my8mcw2fhhsvgtzah7ntr9sdm9zxlamul6uhksyfcvzagemwad7qprplwz
2025-02-10 16:12:31,537 - DEBUG - Token: REDEEM-46eed7 Nonce: 0 Amount: 20000000000000000000000
2025-02-10 16:12:31,537 - DEBUG - Withdraw accepted token
2025-02-10 16:12:31,666 - DEBUG - Contract call arguments: []
The transaction builders are deprecated and will be removed in a later release. Use the transaction factories instead.
2025-02-10 16:12:31,777 - INFO - View this transaction in the MultiversX Devnet Explorer: https://devnet-explorer.multiversx.com/transactions/b965d5d15af80b8558abbf115eaf083be3874e94429a8311f584b5ff2b9f8238


'b965d5d15af80b8558abbf115eaf083be3874e94429a8311f584b5ff2b9f8238'

WITHDRAW TOKENS

# SEND TOKENS

In [10]:
from utils.utils_chain import Account, WrapperAddress
from utils.utils_tx import ESDTToken, multi_esdt_transfer

# sender = Account(pem_file=config.DEFAULT_ACCOUNTS)

# sender.address = WrapperAddress('erd1q3yux0lm7vyutdxyrq7ad762mavurxpuhtuxlwau06eqyr49yknqh6vkk2')
receiver = WrapperAddress('erd1my8mcw2fhhsvgtzah7ntr9sdm9zxlamul6uhksyfcvzagemwad7qprplwz')

tokens = [
    ESDTToken('TKNZPWA-a8b1bc', 0, 10000000000000000000000),
]

deployer.sync_nonce(context.network_provider.proxy)
multi_esdt_transfer(context.network_provider.proxy, 1000000, deployer, receiver, tokens)

2025-02-10 16:11:43,857 - DEBUG - Account.sync_nonce() done: 5603
2025-02-10 16:11:43,858 - DEBUG - Sending multi esdt transfer to erd1my8mcw2fhhsvgtzah7ntr9sdm9zxlamul6uhksyfcvzagemwad7qprplwz
2025-02-10 16:11:43,858 - DEBUG - Args: [<utils.utils_tx.ESDTToken object at 0x7cd95a0b70b0>]
The transaction builders are deprecated and will be removed in a later release. Use the transaction factories instead.
'MultiESDTNFTTransferBuilder' is deprecated and will soon be removed. Please use 'TransferTransactionsFactory' instead.
2025-02-10 16:11:44,079 - INFO - View this transaction in the MultiversX Devnet Explorer: https://devnet-explorer.multiversx.com/transactions/e48fc0e1a157ac853e6c4153ee78ff9205da8874ce68fe183852c7e260021555


'e48fc0e1a157ac853e6c4153ee78ff9205da8874ce68fe183852c7e260021555'